In [4]:
import tensorflow as tf

In [6]:
#Setup for functions
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*x + y + 1

In [8]:
#Run the session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

32


In [9]:
#or
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    res = f.eval()
    print(res)

32


In [10]:
#Reset the node graph in tf
tf.reset_default_graph()

In [11]:
x = tf.constant(3)
y = x + 2
z = x + 1
with tf.Session() as sess:
    res1, res2 = sess.run([y,z])
    print(res1, res2)
    

5 4


In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)


California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.




## Linear regression using the calculated theta function

In [6]:

m,n = housing.data.shape
h_and_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(h_and_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
XtimesXT = tf.matrix_inverse(tf.matmul(XT,X))
theta = tf.matmul(tf.matmul(XtimesXT, XT), y)

with tf.Session() as sess:
    th_val = theta.eval()

## Linear regression computing gradients

In [10]:
from sklearn.preprocessing import StandardScaler
n_epochs = 1000
learning_rate = 0.1
scaler = StandardScaler()
scaled_h_and_bias = scaler.fit_transform(h_and_bias)
X = tf.constant(scaled_h_and_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="error")

#Manually computed gradients
#gradients = 2/m * tf.matmul(tf.transpose(X), error)

#TF computed gradients, prefered
gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for e in range(n_epochs):
        if e % 100 == 0:
            print("Epoch", e, "MSE=", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print("Best theta ", best_theta)

Epoch 0 MSE= 7.5766
Epoch 100 MSE= 4.83399
Epoch 200 MSE= 4.80521
Epoch 300 MSE= 4.80346
Epoch 400 MSE= 4.80328
Epoch 500 MSE= 4.80326
Epoch 600 MSE= 4.80325
Epoch 700 MSE= 4.80325
Epoch 800 MSE= 4.80326
Epoch 900 MSE= 4.80325
Best theta  [[-0.93864417]
 [ 0.82964337]
 [ 0.11875614]
 [-0.26557261]
 [ 0.3057341 ]
 [-0.00450168]
 [-0.03932716]
 [-0.8998304 ]
 [-0.87048841]]
